In [2]:
import pandas as pd
import os
import getopt
import xml.sax
import lxml.sax
import lxml.etree
import re
import json

In [19]:
ids = []
hyperpartisans = []
biases = []

In [12]:
########## SAX FOR STREAM PARSING ##########
class HyperpartisanNewsTFExtractor(xml.sax.ContentHandler):
    def __init__(self):
        xml.sax.ContentHandler.__init__(self)
        self.lxmlhandler = "undefined"

    def startElement(self, name, attrs):
        if name != "articles":
            if name == "article":
                self.lxmlhandler = lxml.sax.ElementTreeContentHandler()

            self.lxmlhandler.startElement(name, attrs)

    def characters(self, data):
        if self.lxmlhandler != "undefined":
            self.lxmlhandler.characters(data)

    def endElement(self, name):
        if self.lxmlhandler != "undefined":
            self.lxmlhandler.endElement(name)
            if name == "article":
                # pass to handleArticle function
                handleArticle(self.lxmlhandler.etree.getroot())
                self.lxmlhandler = "undefined"

In [13]:
########## ARTICLE HANDLING ##########
def handleArticle(article):
    hyperpartisan = article.get("hyperpartisan")
    id_str = article.get("id")
    bias = article.get("bias")

    hyperpartisans.append(hyperpartisan)
    ids.append(id_str)
    biases.append(bias)

In [20]:
with open("data/ground-truth-validation-bypublisher-20181122.xml", 'r') as inputRunFile:
    parser = xml.sax.make_parser()
    parser.setContentHandler(HyperpartisanNewsTFExtractor())
    source = xml.sax.xmlreader.InputSource()
    source.setByteStream(inputRunFile)
    source.setEncoding("utf-8")
    parser.parse(source)
    
    df = pd.DataFrame({"id":ids,
                      "hyperpartisan":hyperpartisans,
                      "bias":biases})
    df.to_pickle("partisanship_annotated_article_ids.pickle")

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
id               150000 non-null object
hyperpartisan    150000 non-null object
bias             150000 non-null object
dtypes: object(3)
memory usage: 3.4+ MB


In [22]:
df.head()

,id,hyperpartisan,bias
0,0000017,false,left-center
1,0000023,true,right
2,0000029,true,left
3,0000032,true,right
4,0000037,false,right-center
